In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import tqdm

/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%pip install optimum[exporters-tf]

Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached tf2onnx-1.16.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached h5py-3.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached grpcio-1.62.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
  Using cached jax-0.4.25-py3-none-any.whl.metadata (24 kB)
  Using cached keras-2.12.0-py2.py3-none-any.w

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification


In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = "Hello Victor "
print(prompt)

In [ ]:
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

In [ ]:
generated_ids = model.generate(**model_inputs, max_new_tokens=20, do_sample=True)
print(tokenizer.batch_decode(generated_ids)[0])

In [3]:
!optimum-cli export tflite --model mistralai/Mistral-7B-v0.1 mistral_tflite

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-packages/optimum/exporters/tflite/__main__.py", line 148, in <module>
    main()
  File "/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-packages/optimum/exporters/tflite/__main__.py", line 61, in main
    model = TasksManager.get_model_from_task(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-packages/optimum/exporters/tasks.py", line 1875, in get_model_from_task
    model_class = TasksManager.get_model_class_for_task(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-packages/optimum/exporters/tasks.py", line 1307, in get_model_class_for_task
    TasksManager._validate_framework_choice(framework)
  File "/opt/conda/envs/mistral_7b_onnx/lib/python3.11/site-pack